In [ ]:
import pandas as pd
from pathlib import Path
import re

In [ ]:
root = "nla"

In [ ]:
name_re = re.compile(r"^batches_(\d+)-exec_(\w+)-matrixfree_(true|false)-size_(\d+)")
time_re = re.compile(r"^Time elapsed:\s+(.+)")

In [ ]:
def parse(file):
    if name_match := name_re.match(file.name):
        with open(file, "r") as f:
            for line in f:
                if time_match := time_re.match(line):
                    return dict(batches=int(name_match.group(1)),
                                size=int(name_match.group(4)),
                                exec=name_match.group(2),
                                matrix_free=name_match.group(3),
                                time=float(time_match.group(1)),
                                )

In [ ]:
parsed_files = []
for file in (Path(root) / "data").iterdir():
    if res := parse(file):
        parsed_files.append(res)
parsed_files

In [ ]:
df = pd.DataFrame(parsed_files)
#df = df.drop(columns=["size"])
df = df.set_index(["batches", "size", "matrix_free"]).sort_index()
df

In [ ]:
df_gpu = df[(df.exec == "hip") | (df.exec == "cuda")]
df_cpu = df[df.exec == "omp"]

In [ ]:
df_gpu.xs(256, level="size")

In [ ]:
def plot_batches(_df):
    size = 256
    _df = _df.xs(size, level="size")
    exec = _df.exec.unique()[0]
    _df.unstack("matrix_free")["time"].plot(logx=True, logy=True, ylabel="Time in [s]", xlabel="Number of batches", title=f"Comparison of matrix-free and matrix-based CG solver [{exec}]")

In [ ]:
plot_batches(df_gpu)

In [ ]:
plot_batches(df_cpu)

In [ ]:
def plot_sizes(_df):
    batches = 100000
    _df = _df.xs(batches, level="batches")
    exec = _df.exec.unique()[0]
    _df.unstack("matrix_free")["time"].plot(logx=True, logy=True, ylabel="Time in [s]", xlabel="Matrix Size", title=f"Comparison of matrix-free and matrix-based CG solver [{exec}]")

In [ ]:
plot_sizes(df_gpu)

In [ ]:
plot_sizes(df_cpu)